# DES	simulation	assignment
## Multiple queues and multiple servers


$\lambda$ – the arrival rate into the system as a whole. \
$\mu$ – the capacity of each of n equal servers. \
$\rho$ represents the system load. \
In a single server system, it will be: $\rho = \frac{\lambda}{\mu}$ \
In a multi-server system (one queue with n equal servers, each with capacity $\mu$), it will be $\rho = \frac{\lambda}{(n \mu)}$

Queuing theory tells us that for FIFO scheduling the average waiting times are shorter for an
M/M/n queue and a system load ρ and processor capacity µ than for a single M/M/1 queue
with the same load characteristics (and thus an n-fold lower arrival rate). Of course, ρ must be
less than one, but the experiment only becomes interesting when ρ is not much less than one.

### 1) Look up and/or derive this theoretical result, at least for n=2. Describe how it is derived. Can you also give a non-mathematical explanation?

TODO

### 2) Write a DES program to verify this for n=1, n=2 and n=4. Make sure that your result has a high and known statistical significance. How does the number of measurements required to attain this depend on ρ?

M/M/n: \
Markovian/memoryless inter-arrival time distribution, so draw from exponential distribution \
Markovian/memoryless service time distribution, so draw from exponential distribution \ 
n servers \
Infinite queue \
Service queuing disciple = FIFO


In [231]:
import numpy as np
import simpy
import matplotlib.pyplot as plt

In [232]:
class Server:
    def __init__(self, env, arrival_rate, service_rate, capacity):
        """
        Initialize the Server object.

        Parameters:
        - env: SimPy environment
        - arrival_rate: Rate of arrivals (requests per time unit)
        - service_rate: Rate of service completion (services per time unit)
        - capacity: Server capacity (number of parallel services allowed)
        """
        self.env = env
        self.server = simpy.Resource(env, capacity=capacity)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.total_arrivals = 0
        self.total_complete_services = 0
        self.total_time_in_queue = 0
        self.arrival_times = []


    def arrival_process(self):
        """
        Process representing the arrival of requests.
        Generates arrivals based on an exponential distribution and
        then starts service processes.
        """
        while True:
            yield self.env.timeout(np.random.exponential(1 / self.arrival_rate))
            self.total_arrivals += 1
            arrival_time = self.env.now
            self.arrival_times.append(arrival_time)
            self.env.process(self.service())


    def service(self):
        """
        Process representing the service of requests.
        Records arrival time, enters the queue, starts service, and calculates time spent in the queue.
        """
        arrival_time = self.env.now
        print(f"Arrival at time {arrival_time}")

        with self.server.request() as req:
            yield req

            service_start_time = self.env.now
            print(f"Service start at time {service_start_time}")
            service_time = np.random.exponential(1 / self.service_rate)
            yield self.env.timeout(service_time)

            departure_time = self.env.now
            print(f"Service end at time {departure_time}")
            self.total_complete_services += 1

            # Calculate time spent in the queue
            time_in_queue = service_start_time - arrival_time
            self.total_time_in_queue += time_in_queue


    def run_simulation(self, simulation_time):
        """
        Run the simulation for a specified duration.
        
        Parameters:
        - simulation_time: max duration of the simulation
        """
        self.env.process(self.arrival_process())
        self.env.run(until=simulation_time)

In [233]:
# Set up simulation environment and parameters
env = simpy.Environment()
arrival_rate = 0.2
service_rate = 0.3
capacity = 1
simulation_time = 100.0 

# Create and run the server simulation
server = Server(env, arrival_rate, service_rate, capacity)
server.run_simulation(simulation_time)

Arrival at time 0.3163367495540208
Service start at time 0.3163367495540208
Service end at time 2.489989964771553
Arrival at time 7.0774788673426965
Service start at time 7.0774788673426965
Arrival at time 7.359406674126699
Service end at time 9.659315194717413
Service start at time 9.659315194717413
Arrival at time 9.858275154372501
Arrival at time 10.086941607818705
Arrival at time 12.262658216238169
Service end at time 14.115194846842853
Service start at time 14.115194846842853
Arrival at time 15.616627677073186
Service end at time 16.795921593966565
Service start at time 16.795921593966565
Service end at time 18.34581387033011
Service start at time 18.34581387033011
Service end at time 19.16274227442708
Service start at time 19.16274227442708
Service end at time 20.96354132159088
Arrival at time 21.7322936671899
Service start at time 21.7322936671899
Arrival at time 23.39089324422032
Arrival at time 23.636486522966024
Service end at time 25.890755138984325
Service start at time 25.

In [234]:
# Print simulation results
print(f"Total Arrivals: {server.total_arrivals}")
print(f"Average Time in Queue: {server.total_time_in_queue / server.total_arrivals if server.total_arrivals > 0 else 0}")
print(f"Total Complete Services: {server.total_complete_services}")

Total Arrivals: 21
Average Time in Queue: 3.0405276678452573
Total Complete Services: 21
